In [1]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
import re
from scipy import stats
from skimage.viewer import ImageViewer
import cv2
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib
import subprocess
import time
import statsmodels.regression.linear_model as sm
from statsmodels.api import OLS



/anaconda/envs/python3/lib/python3.6/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')
/anaconda/envs/python3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from RecSystemKM import RecSystemKM_test

In [3]:

path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df = pd.read_csv(path+'users_followers_merged.csv')

### pull a sample for the pipeline
df = df[2:4]

# Scrape pics from users - don't rerun

for i, v in enumerate(df.username): 
    time.sleep(20)
    subprocess.check_output(["instaloader", (str(v))])
    

# Create the Engagment pipeline

In [4]:


temp_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/All/'
train_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/All/'
save_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/All/'
dest_img =  '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/All/'

userlist= ['or_shmul','fathy_abo_dahis', 'iamkurdapya', 'abhishekraghav015',  'jnbax']
#user list will be the picture number! 


# Vectorize images 

In [5]:
c = RecSystemKM_test(train_path, save_path )
c.Karans_get_train_images(userlist)


#Create Object

train_df = c.convert_to_features(columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"])

Number of images loaded: 302


In [ ]:
# now merge with # of followers

In [6]:
path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df_usernames = pd.read_csv(path+'users_followers_merged.csv')
df_usernames['User_Handle'] = df_usernames.username

#df_usernames['followers'] = (df_usernames['followers'].apply(lambda x: x.replace('k',"000").replace(".","").replace("m","000000").replace(",","")))
#df_usernames.followers = pd.to_numeric(df_usernames.followers)

In [7]:
%%file Engagement_pipeline.py

from sklearn import linear_model

class Engagement:
    
    #Initialization
    def __init__(self, train_path, save_path):
        self.train_path = train_path
        self.save_path = save_path
        
    def zscore(x,mu,std):
        zscore = (x-mu)/std
        return(zscore)

    def metrics(self, df): 
        df[['likes_count','followers']] = df[['likes_count','followers']].apply(pd.to_numeric)

        df['likes_score'] = (df['likes_count'])/(df['followers'])

        df['comments_score'] = (df['comment_count'])/(df['followers'])

        df['zscore_likes'] = zscore(df['likes_score'], df.likes_score.std(), df.likes_score.mean())

        df['zscore_comments'] = zscore(df['comments_score'], df.comments_score.std(), df.comments_score.mean())

        df['final_score'] = df['zscore_likes'] + df['zscore_comments']
        
        return df
        
    def prep_model(self, df, columns_to_remove): 
    
        df = df.drop(columns_to_remove, axis = 1, inplace = True)
        
        rands = np.random.seed(9001)
        msk = np.random.rand(len(df)) < 0.25
        data_train = df[~msk]
        data_test = df[msk]
        
        y_train = data_train.final_score
        y_test = data_test.final_score

        predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
               'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
        X_train = data_train[predictors]
        X_test = data_test[predictors]
        
        return y_train, y_text, X_train, X_test ### should these be self?
        
    def linear(): ## yes those should be self if this is to run automatically
        
        lm = linear_model.LinearRegression() 
        model = lm.fit(X_train,y_train) 
        predictions = lm.predict(X_train)

        model.score(X_train,y_train), model.score(X_test,y_test)
        
    def Ridge():
        alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
        ridge = RidgeCV(alphas=alpha, cv=10, fit_intercept=True)
        ridge.fit(X_train, y_train)
        
        train_score = ridge.score(X_train, y_train)
        test_score = ridge.score(X_test, y_test)
        return(train_score, test_score)



    def Lasso():
        alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
        lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
        lasso.fit(X_train, y_train)
        
        train_score = lasso.score(X_train, y_train)
        test_score = lasso.score(X_test, y_test)
        
        return(train_score, test_score)

        
        ### I also did Poly Feats but seems poor choice here

Overwriting Engagement_pipeline.py


In [8]:


def zscore(x,mu,std):
    zscore = (x-mu)/std
    return(zscore)

def metrics(df): 
    df[['likes_count','followers']] = df[['likes_count','followers']].apply(pd.to_numeric)

    df['likes_score'] = (df['likes_count'])/(df['followers'])
    
    df['comments_score'] = (df['comment_count'])/(df['followers'])
    
    df['zscore_likes'] = zscore(df['likes_score'], df.likes_score.std(), df.likes_score.mean())
    
    df['zscore_comments'] = zscore(df['comments_score'], df.comments_score.std(), df.comments_score.mean())
    
    df['final_score'] = df['zscore_likes'] + df['zscore_comments']
    

In [9]:
df_usernames = df_usernames[0:5]
metrics(df_usernames)

In [61]:
df

,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,ORB_X,ORB_Y,final_score
0,138.5661978298945,68.85854358506768,168.0,133.85562977914478,84.97953749629683,167.0,135.93238711606648,68.48064869294063,155.0,14.965277777777779,127.16971705754598,125.47407576243084,3.521959
1,134.14220225127087,39.89299878365912,136.0,136.18835437225,42.46455028864543,145.0,139.70720662992866,47.352066977158835,152.0,8.80923202614379,183.53733704884849,61.022005335489915,3.521959
2,151.1995119398522,51.268360928596465,165.0,153.80132374300482,54.62034510182328,165.0,155.4477337776069,57.38931896695739,168.0,10.103485838779957,152.41735134124758,110.90057148933411,3.521959
3,143.861954803708,43.83359910090871,157.0,144.2480135845188,43.783710809740576,158.0,142.78141953949336,41.879963576873514,155.0,12.883986928104575,91.61554183959959,72.7517301559448,3.521959
4,152.3564599609375,71.13307536516064,172.0,136.37180419921876,81.45707132690796,150.0,129.059794921875,58.861371280828195,135.0,18.70103759765625,119.66093983769417,120.46904077291488,3.521959
5,169.93778076171876,55.94774983219537,195.0,162.66536865234374,60.42389665519665,187.0,147.715810546875,50.294297687803734,172.0,13.404931640625,121.81377463102342,103.98939337313175,3.521959
6,114.598359375,62.32804919182335,97.0,96.28396484375,64.98113367053654,71.0,83.50236083984375,64.24198400637135,62.0,14.2304443359375,116.57670060396194,119.93656636208296,3.521959
7,164.6656982421875,68.48715507526556,190.0,138.269208984375,78.75741397490727,150.0,128.38324462890625,63.41391742252502,124.0,17.61529541015625,158.42227003991604,163.0114544820786,3.521959
8,136.13362060546876,84.47034430449845,154.0,132.18185546875,76.02384449314684,137.0,127.76057373046875,64.16417538696186,134.0,9.70880126953125,110.4767248606682,159.6415851071477,3.521959
9,139.55376220703124,94.06657710358374,162.0,123.07146728515625,92.99342331387841,107.0,113.6238427734375,69.71795365187482,110.0,16.95289306640625,100.29479246556758,157.6855886435509,3.521959


In [11]:
df = train_df.merge(df_usernames,how='left',on='User_Handle')

In [12]:
# drop extra columns

In [13]:
df.columns

Index(['User_Handle', 'URL', 'R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD',
       'G_MED', 'B_Mean', 'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',
       'Unnamed: 0', 'followers', 'username', 'id_post', 'video_count',
       'url_img', 'link_post', 'owner', 'caption', 'comment_count',
       'taken_at_timestamp', 'taken_at_time', 'shortcode', 'is_video',
       'likes_count', 'likes_score', 'comments_score', 'zscore_likes',
       'zscore_comments', 'final_score'],
      dtype='object')

In [14]:
remove = ['User_Handle', 'URL', 'Unnamed: 0', 'username',  'followers', 'username', 'id_post', 'video_count',
       'url_img', 'link_post', 'owner', 'caption', 'comment_count',
       'taken_at_timestamp', 'taken_at_time', 'shortcode', 'is_video',
       'likes_count', 'likes_score', 'comments_score',
       'zscore_likes', 'zscore_comments',]
df.drop(remove, axis = 1, inplace = True)

In [15]:
## any null?
print(df.isnull().sum(axis=0))
df = df.dropna()
print(df.isnull().sum(axis=0))

R_Mean         0
R_STD          0
R_MED          0
G_Mean         0
G_STD          0
G_MED          0
B_Mean         0
B_STD          0
B_MED          0
Canny          0
ORB_X          0
ORB_Y          0
final_score    4
dtype: int64
R_Mean         0
R_STD          0
R_MED          0
G_Mean         0
G_STD          0
G_MED          0
B_Mean         0
B_STD          0
B_MED          0
Canny          0
ORB_X          0
ORB_Y          0
final_score    0
dtype: int64


# build a model

In [17]:
rands = np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.05
data_train = df[~msk]
data_test = df[msk]

In [19]:
len(data_test)

17

In [20]:
y_train = data_train.final_score
y_test = data_test.final_score

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X_train = data_train[predictors]
X_test = data_test[predictors]

In [30]:
len(y_train), len(X_train)

(220, 220)

In [31]:
for i, v in enumerate(X_train.columns): 
    column = v
    X_train[str(column)] = X_train[str(column)].astype(float)
    


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
y_train = y_train.astype(float)

In [33]:


X_train_constant= sm.add_constant(X_train)
X_test_constant= sm.add_constant(X_test)

#MODEL = sm.OLS(y_train.astype(float), X_train_constant.astype(float)).fit()
#results = model_train.fit()

model = sm.OLS(np.asarray((y_train)), np.asarray((X_train))).fit()
predictions = model.predict(X_train)



In [34]:
from sklearn import linear_model
lm = linear_model.LinearRegression() 
model = lm.fit(X_train,y_train) 
predictions = lm.predict(X_train)

model.score(X_train,y_train), model.score(X_test,y_test)

(0.1439929957948708, -0.04782148068336611)

In [35]:
alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
ridge = RidgeCV(alphas=alpha, cv=10, fit_intercept=True)
ridge.fit(X_train, y_train)
print(ridge.fit(X_train, y_train))
ridge.score(X_train, y_train), ridge.score(X_test, y_test)


RidgeCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], cv=10,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)


(0.1414840209879603, 0.0010873433083060657)

In [36]:
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.1414840209879603, 0.0010873433083060657)

In [37]:


##
lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
print(lasso.fit(X_train, y_train))


LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    copy_X=True, cv=10, eps=0.001, fit_intercept=True, max_iter=10000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)


In [38]:
lasso.score(X_train, y_train), lasso.score(X_test, y_test)

(0.0, -0.0010378470514764704)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

for i,v in enumerate([1,2,3,4,5]): 
    
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, LinearRegression())
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)


In [40]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

for i,v in enumerate([1,2,3,4,5]):     
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, lasso)
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)


1 train :  0.0
1 test :  -0.0010378470514764704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


2 train :  0.020577097385743093
2 test :  0.03522576768893704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


3 train :  0.6211280301068582
3 test :  -0.7090386256544925


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


KeyboardInterrupt: 

# DT, Lasso, Ridge

In [ ]:
#### WP NOTES 

from sklearn.linear_model import Lasso, Ridge

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor


y = df.final_score

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X = df[predictors]

#X = normalize(X, axis=0)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
print(X.shape)

poly = PolynomialFeatures(4) 
X = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=9)
print(X_train.shape)

#l_model = Lasso(alpha=0.001)
l_model = DecisionTreeRegressor(max_depth=2)
l_model.fit(X_train, y_train)
print(y_train)
print(l_model.score(X_train, y_train))
print(l_model.score(X_test, y_test))

In [59]:
#### WP NOTES 

from sklearn.linear_model import Lasso, Ridge

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import normalize 


y = df.final_score

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X = df[predictors]

X = normalize(X, axis=0)
print(X.shape)

poly = PolynomialFeatures(4) 
X = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(X_train.shape)

l_model = Lasso(alpha=10.)
l_model.fit(X_train, y_train)
print(l_model.score(X_train, y_train))
print(l_model.score(X_test, y_test))

(297, 12)
(178, 1820)
265   -0.660599
174    3.521959
26     3.521959
2      3.521959
21     3.521959
13     3.521959
116    3.521959
162    3.521959
131    3.521959
148    3.521959
86     3.521959
175    3.521959
273   -0.215174
28     3.521959
55     3.521959
47     3.521959
190    3.521959
107    3.521959
293   -0.215174
25     3.521959
97     3.521959
146    3.521959
79     3.521959
11     3.521959
142    3.521959
138    3.521959
225    3.521959
283   -0.215174
67     3.521959
189    3.521959
         ...   
285   -0.215174
205    3.521959
58     3.521959
141    3.521959
49     3.521959
208    3.521959
137    3.521959
119    3.521959
227    3.521959
129    3.521959
184    3.521959
214    3.521959
274   -0.215174
12     3.521959
120    3.521959
234    3.521959
187    3.521959
238    3.521959
221    3.521959
117    3.521959
248    3.521959
241    3.521959
250    3.521959
59     3.521959
22     3.521959
65     3.521959
125    3.521959
251    3.521959
56     3.521959
126    3.521959
Na

In [ ]:
#### WP NOTES 

from sklearn.linear_model import Lasso, Ridge

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import normalize 


y = df.final_score

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X = df[predictors]

X = normalize(X, axis=0)
print(X.shape)

poly = PolynomialFeatures(4) 
X = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(X_train.shape)

l_model = Ridge(alpha=10.)
l_model.fit(X_train, y_train)
print(l_model.score(X_train, y_train))
print(l_model.score(X_test, y_test))

In [44]:
lasso.mse_path_  ## mse to r^2  --> this is for the ten fold 

array([[ 1.36932041,  0.54743859,  2.08215054,  0.9218996 ,  1.35489945,
         4.5410607 ,  1.35229693,  6.34075758,  5.55157387, 11.13021377],
       [ 2.09952354,  1.00761317,  4.55590726,  4.83921486,  2.65081373,
        10.23757292,  2.39028854,  8.08110628,  6.17255905, 14.43877083],
       [ 3.78202257,  1.34615864,  4.33469987,  6.31072484,  4.77796531,
        12.7864636 ,  4.2435499 ,  8.64824713,  6.16172524, 22.50252867],
       [ 5.48213297,  1.73116615,  4.30778818,  7.20135006,  6.29238961,
        13.96667881,  6.57457755, 11.40166137,  6.54458065, 29.88135656],
       [ 7.07134538,  1.86377504,  4.24083359,  7.036417  ,  6.99955422,
        14.77109688,  7.95823933, 12.99982475,  7.15166986, 30.33783082],
       [ 8.31970964,  1.93666605,  4.28037181,  7.33327395,  7.47213781,
        15.60594864,  8.81226824, 13.57803514,  7.64750211, 31.19737441],
       [ 9.45921245,  2.00052206,  4.27941434,  7.97283969,  8.05708828,
        16.57541683,  9.42493848, 13.42502021

In [ ]:
for i,v in enumerate([1,2,3,4,5]):     
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, ridge)
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)
